In [3]:
# Author: Jaehun Kim
# Email: rlawogns1204@unist.ac.kr
# Affiliation: UNIST BME BCILAB
# Date: 2023-07-14
#
# This code implements a tactile information processing model using a spiking
# neural network (SNN). It simulates the processing of tactile information from
# mechanoreceptors in the skin through primary afferent fibers (PA), cuneate nucleus
# neurons (PN and IN), and ultimately, somatosensory cortex neurons. The model
# incorporates lateral inhibition and various receptive field properties to
# represent a realistic processing of touch stimuli.

# with DIGIT-sensor
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import scipy.io
import time
import numpy as np
from PIL import Image
import io
import datetime
import cv2
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from tqdm import tqdm

from Function.snn_IZHIlayers import *
from Function.snn_plot import *
from Function.snn_simulation import *
from Function.snn_stimulation import *
from Function.snn_receptive_field_weights import *

from line_profiler import LineProfiler

%matplotlib inline
%load_ext autoreload
%autoreload 2

print(f"GPU available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = 'cpu'
print(f"Using device: {device}")\

# Set sensor dimensions (height and width in millimeters)
sensor_h, sensor_w = 19, 16
# Set pixel dimensions (number of pixels in height and width)
# pixel_h, pixel_w = 320, 240///
pixel_h, pixel_w = 64,48
# Set image frames per second (FPS) of tactile sensor
image_FPS = 60

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
GPU available: False
CUDA version: None
Using device: cpu


In [4]:
R = ReceptiveFieldWeights(pixel_h, pixel_w, device, type_output = [np.pi/2], plot_receptive_field = False,plot_ind = 1)

[1.5707963267948966]
torch.Size([88, 3072])
sa_rf shape: torch.Size([88, 3072]) with height = 11 with width = 8
ra_rf shape: torch.Size([130, 3072]) with height = 13 with width = 10
sa_cn_pn_rf shape:  torch.Size([54, 88]) sa_cn_pn_step_height: 9 sa_cn_pn_step_width: 6
sa_cn_in_rf shape:  torch.Size([54, 88]) sa_cn_in_step_height: 9 sa_cn_in_step_width: 6
ra_cn_pn_rf shape:  torch.Size([88, 130]) ra_cn_pn_step_height: 11 ra_cn_pn_step_width: 8
ra_cn_in_rf shape:  torch.Size([88, 130]) ra_cn_in_step_height: 11 ra_cn_in_step_width: 8
sa_intopn_rf shape:  torch.Size([54, 54])
ra_intopn_rf shape:  torch.Size([88, 88])
cn_pn_sa_rf shape:  torch.Size([28, 54]) cn_pn_sa_rf_step_height: 7 cn_pn_sa_rf_step_width: 4
cn_in_sa_rf shape:  torch.Size([28, 54]) cn_in_sa_rf_step_height: 7 cn_in_sa_rf_step_width: 4
cn_pn_ra_rf shape:  torch.Size([28, 88]) cn_pn_ra_rf_step_height: 7 cn_pn_ra_rf_step_width: 4
cn_in_ra_rf shape:  torch.Size([28, 88]) cn_in_ra_rf_step_height: 7 cn_in_ra_rf_step_width: 4
cn

In [5]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
import numpy as np

num_trials = 20
angles = range(0, 180, 5) # Angles from 0 to 175 with a step of 5
num_stim = 300
noise_std_values = range(1,13, 1)
window_sizes = range(1, 270, 20)  # Window sizes from 0 to 200 with a step of 10

# Initialize an accuracy matrix to store the accuracy for each combination of window size and noise_std
accuracy_matrix = np.zeros((len(window_sizes), len(noise_std_values)))

plot_figure = 0 

# Iterate over all combinations of window size and noise_std values
for j, noise_std in enumerate(noise_std_values):
    # Initialize empty lists to store data and labels
    data_per_noise = []
    labels_per_noise = []
    # Run the trials
    for trial in range(num_trials):
        # Generate the stimuli for each angle
        for k, angle in enumerate(angles):
            print(f", noise_std: {noise_std} - Running trial {trial+1}, num_stim: {num_stim}, angle: {angle}")
            # Generate the stimulus
            stim =generate_stimuli(angle=angle, num_stim=num_stim, pixel_h = 64, pixel_w=48, F=10, plot_stimuli=False, device=device)

            S = SNN(R, device = device, noise_std_val = noise_std)  # Update with your SNN initialization method
            S.feedforward(stim)
            spike_times = S.cn_spike_times[1]

            # Save the spike_times and labels
            data_per_noise.append(spike_times.cpu().numpy())
            labels_per_noise.append(angle)

    # Convert spike_times and labels to numpy arrays
    spike_times = np.array(data_per_noise)
    labels = np.array(labels_per_noise)

    for i, window_size in enumerate(window_sizes):
        # Compute firing rates for each window size
        data = []
        for spike_time in spike_times:
            mean_firing_rate = compute_individual_firing_rates(spike_time, window_size)
            data.append(mean_firing_rate)

        # Split the data into training and test sets
        X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

        # Create and train the SVM
        clf = svm.SVC(kernel='linear')  # You can change the kernel as needed
        clf.fit(X_train, y_train)

        # Test the SVM
        y_pred = clf.predict(X_test)

        # Get the accuracy
        accuracy = accuracy_score(y_test, y_pred)

        # Store the accuracy in the accuracy matrix
        accuracy_matrix[i, j] = accuracy

        # Print the accuracy
        print(f"For window size {window_size}, noise_std {noise_std} and angle {angle}, Accuracy: ", accuracy)

# Print the final accuracy matrix
print("Accuracy matrix: ", accuracy_matrix)


, noise_std: 1 - Running trial 1, num_stim: 300, angle: 0


c:\Users\JaehunKim\anaconda3\envs\pytorch-cpu-3.11\Lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


, noise_std: 1 - Running trial 1, num_stim: 300, angle: 5
, noise_std: 1 - Running trial 1, num_stim: 300, angle: 10
, noise_std: 1 - Running trial 1, num_stim: 300, angle: 15
, noise_std: 1 - Running trial 1, num_stim: 300, angle: 20
, noise_std: 1 - Running trial 1, num_stim: 300, angle: 25
, noise_std: 1 - Running trial 1, num_stim: 300, angle: 30
, noise_std: 1 - Running trial 1, num_stim: 300, angle: 35
, noise_std: 1 - Running trial 1, num_stim: 300, angle: 40
, noise_std: 1 - Running trial 1, num_stim: 300, angle: 45
, noise_std: 1 - Running trial 1, num_stim: 300, angle: 50
, noise_std: 1 - Running trial 1, num_stim: 300, angle: 55
, noise_std: 1 - Running trial 1, num_stim: 300, angle: 60
, noise_std: 1 - Running trial 1, num_stim: 300, angle: 65
, noise_std: 1 - Running trial 1, num_stim: 300, angle: 70
, noise_std: 1 - Running trial 1, num_stim: 300, angle: 75
, noise_std: 1 - Running trial 1, num_stim: 300, angle: 80
, noise_std: 1 - Running trial 1, num_stim: 300, angle: 8

In [ ]:
import matplotlib.cm as cm
import matplotlib as mpl

print("Accuracy matrix: ", accuracy_matrix)

plt.figure(figsize=(10, 6))
colors = cm.viridis(np.linspace(0, 1, len(noise_std_values)))

for i, noise_std in enumerate(noise_std_values):
    plt.plot(window_sizes, [row[i] for row in accuracy_matrix], color=colors[i])

plt.xlabel('Window size')
plt.ylabel('Accuracy')
plt.grid(True)
plt.title('Accuracy for different window sizes and noise std values')

# Colorbar
cax = plt.axes([0.93, 0.1, 0.03, 0.8])
norm = mpl.colors.Normalize(vmin=min(noise_std_values), vmax=max(noise_std_values))
sm = plt.cm.ScalarMappable(cmap=cm.viridis, norm=norm)
sm.set_array([])
plt.colorbar(sm, cax=cax)

plt.show()

# Create a descriptive filename
filename = f"accuracy_matrix_trials{num_trials}_angles{min(angles)}to{max(angles)}_stim{num_stim}_noise{min(noise_std_values)}to{max(noise_std_values)}_window{min(window_sizes)}to{max(window_sizes)}.npy"

# Save the file
np.save(filename, accuracy_matrix)
